In [1]:
import re
import string
import torch
from torch import nn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
# import gensim.downloader
from gensim.models import Word2Vec
import scipy
import torch
from torch import nn
from torch import optim
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader

In [2]:
# model = gensim.downloader.load('glove-twitter-25')

In [3]:
df = pd.read_csv('twitter_cleaned.csv')
df = df.dropna()

In [4]:
tokenized_tweet = df['message'].apply(lambda x: x.split())


In [5]:
%%time
model = Word2Vec(
            tokenized_tweet,
            vector_size=200, # desired no. of features/independent variables
            window=5, # context window size
            min_count=2, # Ignores all words with total frequency lower than 2.                                  
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 32, # no.of cores
            seed = 34
) 

model.train(tokenized_tweet, total_examples= len(df['message']), epochs=20)

CPU times: total: 8min 22s
Wall time: 1min 23s


(14292250, 19171820)

In [6]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model.wv[word].reshape((1, size))
            count += 1.
        except KeyError:  # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec

In [7]:
wordvec_arrays = np.zeros((len(tokenized_tweet), 200)) 
for i in range(len(tokenized_tweet)):
    try:
            wordvec_arrays[i,:] = word_vector(tokenized_tweet[i], 200)
    except KeyError:  # handling the case where the token is not in vocabulary
            continue
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

(49675, 200)

In [8]:
wordvec_df.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.021858,-0.136008,-0.079867,0.142262,-0.334975,0.006666,-0.085440,0.008387,0.076322,-0.042974,...,0.267254,-0.212343,0.075734,0.067654,0.181160,-0.150075,-0.048985,-0.284484,0.137764,0.179570
1,-0.100456,-0.260915,-0.010394,-0.032613,-0.222945,-0.084207,-0.161302,-0.160732,0.091418,-0.080268,...,0.126912,-0.262101,0.061075,-0.092836,0.134562,0.088054,-0.052204,-0.246774,0.008476,-0.082413
2,0.108536,-0.243100,0.025253,-0.038388,-0.250112,0.014699,-0.217285,0.007953,0.193621,0.039194,...,0.035768,-0.247438,0.149372,0.052816,0.101603,0.206118,-0.017807,-0.195487,0.170776,-0.070792
3,0.019540,-0.273690,-0.107766,0.043199,-0.260629,-0.081694,0.058818,-0.201367,0.114887,-0.027537,...,0.125935,-0.374731,-0.008154,-0.048901,0.117185,-0.010782,-0.039707,-0.268269,0.148644,-0.005968
4,-0.001681,-0.189922,-0.008858,-0.030504,-0.193903,-0.117119,-0.157301,-0.092671,0.145255,-0.025514,...,0.193059,-0.213469,0.078424,-0.056320,0.095797,0.186037,-0.047434,-0.275176,-0.010689,0.007353


In [9]:
wordvec_df.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.021858,-0.136008,-0.079867,0.142262,-0.334975,0.006666,-0.085440,0.008387,0.076322,-0.042974,...,0.267254,-0.212343,0.075734,0.067654,0.181160,-0.150075,-0.048985,-0.284484,0.137764,0.179570
1,-0.100456,-0.260915,-0.010394,-0.032613,-0.222945,-0.084207,-0.161302,-0.160732,0.091418,-0.080268,...,0.126912,-0.262101,0.061075,-0.092836,0.134562,0.088054,-0.052204,-0.246774,0.008476,-0.082413
2,0.108536,-0.243100,0.025253,-0.038388,-0.250112,0.014699,-0.217285,0.007953,0.193621,0.039194,...,0.035768,-0.247438,0.149372,0.052816,0.101603,0.206118,-0.017807,-0.195487,0.170776,-0.070792
3,0.019540,-0.273690,-0.107766,0.043199,-0.260629,-0.081694,0.058818,-0.201367,0.114887,-0.027537,...,0.125935,-0.374731,-0.008154,-0.048901,0.117185,-0.010782,-0.039707,-0.268269,0.148644,-0.005968
4,-0.001681,-0.189922,-0.008858,-0.030504,-0.193903,-0.117119,-0.157301,-0.092671,0.145255,-0.025514,...,0.193059,-0.213469,0.078424,-0.056320,0.095797,0.186037,-0.047434,-0.275176,-0.010689,0.007353


In [10]:
# NORMALIZING

wordvec_df = (wordvec_df - wordvec_df.min()) / (wordvec_df.max() - wordvec_df.min())
wordvec_df.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.571994,0.623757,0.309427,0.718099,0.373818,0.508229,0.503443,0.526289,0.426138,0.476286,...,0.647968,0.610293,0.517064,0.644147,0.564423,0.245045,0.510967,0.407351,0.508058,0.647381
1,0.462480,0.471853,0.388536,0.531941,0.497407,0.382798,0.434003,0.361507,0.443034,0.431975,...,0.501646,0.554037,0.499443,0.492848,0.516364,0.513032,0.507250,0.445175,0.381707,0.315725
2,0.649602,0.493519,0.429128,0.525793,0.467436,0.519316,0.382759,0.525866,0.557426,0.573915,...,0.406618,0.570615,0.605580,0.630158,0.482372,0.645900,0.546970,0.496618,0.540319,0.330437
3,0.569918,0.456317,0.277657,0.612645,0.455834,0.386266,0.635489,0.321914,0.469302,0.494628,...,0.500627,0.426700,0.416226,0.534267,0.498443,0.401804,0.521681,0.423616,0.518691,0.412501
4,0.550918,0.558191,0.390285,0.534186,0.529445,0.337371,0.437665,0.427822,0.503292,0.497031,...,0.570611,0.609020,0.520297,0.527273,0.476385,0.623301,0.512758,0.416688,0.362978,0.429364


In [11]:
x_train, x_test, y_train, y_test = train_test_split(
    wordvec_df.squeeze(), df['category'], test_size=.2, stratify=df['label'], random_state=128)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(39740, 200) (9935, 200) (39740,) (9935,)


In [12]:
x_train = torch.tensor(x_train.values).float()
x_test = torch.tensor(x_test.values).float()


In [13]:
y_train = torch.tensor(y_train.values)
y_test = torch.tensor(y_test.values)

In [14]:
def topk_encoding(nd_array):
    predictions = nd_array
    
    ps = torch.exp(predictions)
    top_p, top_class  = ps.topk(1, dim=1)
    

    return top_class

In [25]:
class NeuralNetwork(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        
        self.hidden_layer_1 = nn.Linear(x_train.shape[1], 64) # input to first hidden layer
        self.output_layer = nn.Linear(64, self.out_dim)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        x = self.hidden_layer_1(x)
        x = self.activation(x)
        x = self.dropout(x)
        
        y = self.output_layer(x)
        y = self.activation(y)
        y = self.softmax(y)
        
        return y

In [26]:
model_neural = NeuralNetwork(x_train.shape[1], df['category'].nunique())
# model = NeuralNetwork(x_train.shape[1], 5)


# Define the loss
criterion = nn.NLLLoss()

# Optimizers require the parameters to optimize and a learning rate
optimizer = optim.Adam(model_neural.parameters(), lr=0.002)

#setting up scheduler
# scheduler = ReduceLROnPlateau(optimizer, 'min', patience = 10)

In [31]:
%%time
train_losses = []
test_losses = []
test_accuracies = []

epochs = 100
for e in range(epochs):
    optimizer.zero_grad()

    output = model_neural.forward(x_train)
    loss = criterion(output, y_train)
    loss.backward()
    train_loss = loss.item()
    train_losses.append(train_loss)
    
    optimizer.step()

    # Turn off gradients for validation, saves memory and computations
    with torch.no_grad():
        model_neural.eval()
        log_ps = model_neural.forward(x_test)
        test_loss = criterion(log_ps, y_test)
        test_losses.append(test_loss)

        ps = torch.exp(log_ps)
        top_p, top_class  = ps.topk(1, dim=1)
        equals = top_class == y_test.view(*top_class.shape)
        test_accuracy = torch.mean(equals.float())
        test_accuracies.append(test_accuracy)

    model_neural.train()

    print(f"Epoch: {e+1}/{epochs}.. ",
          f"Training Loss: {train_loss:.3f}.. ",
          f"Test Loss: {test_loss:.3f}.. ",
          f"Test Accuracy: {test_accuracy:.3f}")
#     scheduler.step(test_loss/len(y_test))

Epoch: 1/100..  Training Loss: 1.003..  Test Loss: 1.008..  Test Accuracy: 0.471
Epoch: 2/100..  Training Loss: 1.004..  Test Loss: 1.008..  Test Accuracy: 0.471
Epoch: 3/100..  Training Loss: 1.003..  Test Loss: 1.008..  Test Accuracy: 0.470
Epoch: 4/100..  Training Loss: 1.003..  Test Loss: 1.008..  Test Accuracy: 0.469
Epoch: 5/100..  Training Loss: 1.003..  Test Loss: 1.008..  Test Accuracy: 0.470
Epoch: 6/100..  Training Loss: 1.003..  Test Loss: 1.008..  Test Accuracy: 0.469
Epoch: 7/100..  Training Loss: 1.003..  Test Loss: 1.008..  Test Accuracy: 0.470
Epoch: 8/100..  Training Loss: 1.003..  Test Loss: 1.008..  Test Accuracy: 0.470
Epoch: 9/100..  Training Loss: 1.003..  Test Loss: 1.008..  Test Accuracy: 0.469
Epoch: 10/100..  Training Loss: 1.003..  Test Loss: 1.008..  Test Accuracy: 0.469
Epoch: 11/100..  Training Loss: 1.003..  Test Loss: 1.008..  Test Accuracy: 0.469
Epoch: 12/100..  Training Loss: 1.003..  Test Loss: 1.008..  Test Accuracy: 0.469
Epoch: 13/100..  Training

In [34]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [32]:
preds = model_neural(x_test)
preds = topk_encoding(preds)
preds

tensor([[1],
        [1],
        [1],
        ...,
        [2],
        [2],
        [1]])

In [35]:
acc_score = accuracy_score(y_test, preds)

acc_score

0.4638147961751384

In [36]:
f1_score = f1_score(y_test, preds, average='macro')
f1_score

0.32795210974961847

In [21]:
# # Custom dataset to treat how the model picks an x, y combination from the dataset
# class MyCustomDataset(Dataset):
#     def __init__(self, x, y):
#         self.x = x
#         self.y = y
        
#     def __len__(self):
#         return len(self.x)
    
#     # Requires you to return data as a pair of _x, _y
#     def __getitem__(self, index):
#         return self.x[index], self.y[index]

In [22]:
# custom_dataset_train = MyCustomDataset(x=x_train, y=y_train)
# custom_dataset_test = MyCustomDataset(x=x_test, y=y_test)

In [23]:
# train_loader = DataLoader(
#     custom_dataset_train,
#     batch_size=16,
#     shuffle=False,
#     drop_last=False
# )

# test_loader = DataLoader(
#     custom_dataset_test,
#     batch_size=16,
#     shuffle=False,
#     drop_last=False
# )

In [24]:
# %%time
# train_losses = []
# test_losses = []
# test_accuracies = []

# epochs = 50
# for e in range(epochs):
    
#     for data, label in train_loader:
    
#         optimizer.zero_grad()

#         output = model_neural.forward(data)
#         loss = criterion(output, label)
#         loss.backward()
#         train_loss = loss.item()
#         train_losses.append(train_loss)
    
#         optimizer.step()

#     # Turn off gradients for validation, saves memory and computations
#     with torch.no_grad():
#         model_neural.eval()
#         for data, label in test_loader:
#             log_ps = model_neural.forward(data)
#             test_loss = criterion(log_ps, label)
#             test_losses.append(test_loss)

#             ps = torch.exp(log_ps)
#             top_p, top_class  = ps.topk(1, dim=1)
#             equals = top_class == label.view(*top_class.shape)
#             test_accuracy = torch.mean(equals.float())
#             test_accuracies.append(test_accuracy)
#         model_neural.train()

#         print(f"Epoch: {e+1}/{epochs}.. ",
#               f"Training Loss: {train_loss:.3f}.. ",
#               f"Test Loss: {test_loss:.3f}.. ",
#               f"Test Accuracy: {test_accuracy:.3f}")
#         scheduler.step(test_loss/len(y_test))